## Stacked Generalization

The procedure for a 5 fold stacking may be described as follows:

1. Split the total training set into two disjoint sets (here train and holdout)

2. Train several base models on the first part (train)

3. Predict these base models on the second part (holdout)

4. Repeat step 1-3 five times and use the holdout predictions as the inputs, and the correct responses (target variable) as the outputs to train a higher level learner called meta-model.


- For the test set, we could either average the predictions of all base models on the test data or refit the model using the whole training set and then predict. Generally speaking, either way is fine because the test set hasn't seen the training set.
- If we ran 10 models using the same procedure, our meta model will have 10 input features.

![img](https://s3.amazonaws.com/nycdsabt01/stacking.jpg)

Borrowed from [Faron](https://www.kaggle.com/getting-started/18153#post103381)

As a quick note, one should try a few diverse models. To my experience, a good stacking solution is often composed of at least:
- 2 or 3 GBMs/XGBs/LightGBMs (one with low depth, one with medium and one with high)
- 1 or 2 Random Forests (again as diverse as possible–one low depth, one high)
- 1 linear model

In [54]:
from sklearn.ensemble import VotingClassifier

In [69]:
from sklearn.linear_model import ElasticNet, LinearRegression as lr
from sklearn.ensemble import GradientBoostingRegressor as gbr, RandomForestRegressor as rfr
from xgboost import XGBRegressor

In [56]:
# Useful if you are debugging the function inside another .py script
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import pandas as pd

houses_train = pd.read_csv('../Data/encoded_houses_train.csv')
houses_test = pd.read_csv('../Data/encoded_houses_test.csv')

In [58]:
houses_test.SalePrice.head()

0    0
1    0
2    0
3    0
4    0
Name: SalePrice, dtype: int64

In [59]:
X_train = houses_train.loc[:, houses_train.columns != "SalePrice"].values # convert to np.array
y_train = houses_train.loc[:, houses_train.columns == "SalePrice"].values.reshape(-1, )

X_test = houses_test.loc[:, houses_train.columns != "SalePrice"].values # convert to np.array

In [60]:
from stacking import stacking_regression
from sklearn.metrics import mean_squared_error
import numpy as np

In [61]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(np.log(y), np.log(y_pred)))

In [78]:
models = [
    # linear model, ElasticNet = lasso + ridge
    ElasticNet(random_state=0, 
              fit_intercept=True, alpha=0.18069, l1_ratio=0.01),
    
    # deep random forst model
    rfr(random_state=0,
        n_estimators=1000, max_depth=20,  max_features=70),
    
    # aggressive random forst model
    rfr(random_state=0, 
        n_estimators=1500, max_depth=10,  max_features=75),
    
    # conservative gbm model
    gbr(random_state=0, learning_rate = 0.005, max_features='sqrt',
        min_samples_leaf=15, min_samples_split=10, 
        n_estimators=3000, max_depth=3),
    
    # aggressive gbm model
    gbr(random_state = 0, learning_rate = 0.01, max_features='sqrt',
        min_samples_leaf=10, min_samples_split=5, 
        n_estimators = 1000, max_depth = 9),
    
    XGBRegressor(max_depth=3, 
                 learning_rate=0.03, 
                 n_estimators=1700, # Number of boosted trees to fit
                 silent=True, # print messages while running 
                 objective='reg:linear', 
                 booster='gbtree', # Specify which booster to use: gbtree, gblinear or dart
                 n_jobs=-1, # Number of parallel threads used to run xgboost. (replaces nthread)
                 gamma=0,  # Minimum loss reduction required to make a further partition on a leaf node of the tree.
                 min_child_weight=1, # Minimum sum of instance weight(hessian) needed in a child
                 max_delta_step=0, # Maximum delta step we allow each tree’s weight estimation to be
                 subsample=1, # Subsample ratio of the training instance
                 colsample_bytree=1, # Subsample ratio of columns when constructing each tree
                 colsample_bylevel=0.3, # Subsample ratio of columns for each split, in each level
                 reg_alpha=0, # L1 regularization term on weights
                 reg_lambda=1, # L2 regularization term on weights
                 scale_pos_weight=1, # Balancing of positive and negative weights
                 base_score=0.5, # The initial prediction score of all instances, global bias
                 random_state=743, 
                 missing=None)       
    ]

meta_model = lr(normalize=True)

In [130]:
%%time
stacking_prediction = stacking_regression(models, 
                                          rfr(random_state=0,
                                              n_estimators=1000, 
                                              max_depth=20,  
                                              max_features=2), 
                                          X_train, y_train, X_test,
                                          transform_target=np.log1p, transform_pred = np.expm1, 
                                          metric=rmsle, verbose=2)

metric: [rmsle]

model 0: [ElasticNet]
    fold 0: [0.12733265]
    fold 1: [0.15022274]
    fold 2: [0.13589971]
    ----
    MEAN:   [0.13814301]

model 1: [RandomForestRegressor]
    fold 0: [0.13083948]
    fold 1: [0.15361229]
    fold 2: [0.12955067]
    ----
    MEAN:   [0.13844852]

model 2: [RandomForestRegressor]
    fold 0: [0.13149321]
    fold 1: [0.15441378]
    fold 2: [0.13017531]
    ----
    MEAN:   [0.13914579]

model 3: [GradientBoostingRegressor]
    fold 0: [0.11873860]
    fold 1: [0.14330742]
    fold 2: [0.11815123]
    ----
    MEAN:   [0.12727946]

model 4: [GradientBoostingRegressor]
    fold 0: [0.11989911]
    fold 1: [0.14375247]
    fold 2: [0.11856115]
    ----
    MEAN:   [0.12793496]

model 5: [XGBRegressor]
    fold 0: [0.11341232]
    fold 1: [0.13488918]
    fold 2: [0.11454766]
    ----
    MEAN:   [0.12135610]

CPU times: user 1min 20s, sys: 546 ms, total: 1min 20s
Wall time: 1min 21s


In [131]:
stacking_prediction[100:105]
stacking_prediction.shape
stacking_prediction.mean(axis=1).shape

array([[ 127105.55992651,  136655.35847707,  135104.37020173,
         133678.10663924,  132143.12258639,  146018.82291667],
       [ 137412.03036976,  130008.91464806,  130594.41255991,
         134093.65574022,  132753.1569182 ,  128873.3125    ],
       [ 120600.16536177,  125999.55473139,  125837.58377347,
         120684.7373917 ,  123356.66034635,  120459.921875  ],
       [ 152045.71243471,  152824.85540069,  153856.58421047,
         152363.4550493 ,  152477.38267359,  146918.74479167],
       [ 141032.96167964,  149500.33968225,  149712.88825911,
         148565.14198989,  151014.43535457,  144769.609375  ]])

(1459, 6)

(1459,)

In [133]:
stacking_prediction.mean(axis=1)[1458]

214281.13531258886

In [129]:
y_train[1459]
y_train.shape

147500.0

(1460,)

In [76]:
stacking_prediction

array([ 123284.27605339,  146191.75888207,  175551.22032681, ...,
        146172.25115723,  121320.4997873 ,  194930.93656124])

In [103]:
stacking_prediction

array([ 117727.38857337,  149803.51894442,  184443.48134752, ...,
        155993.94550037,  108077.20099528,  215095.8125255 ])

**Having more models than necessary in ensemble may hurt.**

Lets say we have a library of created models. Usually greedy-forward approach works well:
- Start with a few well-performing models’ ensemble
- Loop through each other model in a library and add to current ensemble
- Determine best performing ensemble configuration
- Repeat until metric converged

If you are using linear regression as the meta model, make sure you have **diverse/uncorrelated** first layer models

During each loop iteration it is wise to consider only a subset of library models, which could work as a regularization for model selection.

Repeating procedure few times and bagging results reduces the possibility of overfitting by doing model selection.

Another [great walkthrough](https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python) of stacking on Kaggle using the famous Titanic dataset.

R users can call the `stackedEnsemble()` function from the [H2o package](https://h2o-release.s3.amazonaws.com/h2o/rel-ueno/2/docs-website/h2o-docs/data-science/stacked-ensembles.html) directly.

### Success formula (personal opinion)

50% - feature engineering

30% - model diversity

10% - luck

10% - proper ensembling
 - Voting
 - Averaging
 - Stacking

In [134]:
submission = pd.DataFrame({"Id": range(1461, 2920), 
                            "SalePrice": stacking_prediction.mean(axis=1)})   # values

In [135]:
import datetime
time = '{:%Y-%m-%d %H %M}'.format(datetime.datetime.now())
time

'2017-11-11 18 59'

In [136]:
submission.to_csv("./Ensemble Submissions/submission {}.csv".format(time), sep=',', index = False)